# Ciclismo CDMX

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
contador = pd.read_csv('Data/contador-ciclistas.csv')
incidentes = pd.read_csv('Data/incidentes-viales-c5.csv')
estudio = pd.read_csv('Data/estudio-de-conteo-ciclista-2018.csv')

In [3]:
contador.shape

(2374, 7)

In [4]:
contador.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2374 entries, 0 to 2373
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Día de la semana           2374 non-null   object 
 1   Día                        2374 non-null   int64  
 2   Mes                        2374 non-null   object 
 3   Año                        2374 non-null   int64  
 4   Contador Ruta Reforma222   2150 non-null   float64
 5   Contador Ruta Revolución   1382 non-null   float64
 6   Contador Ruta Patriotismo  1386 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 130.0+ KB


## Contador ciclista de Reforma, Revolución y Patriotismo

Conteo de Revolución y Patriotismo se inició el 14 de junio de 2015.

In [5]:
meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
contador['Mes'] = pd.Categorical(contador['Mes'], meses)
contador = contador.sort_values(['Año', 'Mes', 'Día'])
contador['Mes'] = contador['Mes'].astype('object')

In [6]:
dict_meses = {}
number = 0
for mes in meses: 
    number += 1
    dict_meses[mes] = number

In [7]:
contador['Mes'] = contador['Mes'].map(dict_meses)

In [8]:
contador.insert(4, 'Fecha', pd.to_datetime(contador.Año*10000+contador.Mes*100+contador.Día, format='%Y%m%d'))

In [9]:
contador = contador.reset_index(drop=True)

In [10]:
dict_meses_inverso = {}
for mes, numero in dict_meses.items():
    dict_meses_inverso[numero] = mes

In [11]:
contador['Mes'] = contador['Mes'].map(dict_meses_inverso)

In [12]:
contador['Día de la semana'] = contador['Día de la semana'].str.replace('á', 'a').str.replace('é', 'e')

In [13]:
contador['Contador Ruta Reforma222'] = contador['Contador Ruta Reforma222'].fillna(0)
contador['Contador Ruta Revolución'] = contador['Contador Ruta Revolución'].fillna(0)
contador['Contador Ruta Patriotismo'] = contador['Contador Ruta Patriotismo'].fillna(0)

In [14]:
contador['Contador Ruta Reforma222'] = contador['Contador Ruta Reforma222'].astype('int64')
contador['Contador Ruta Revolución'] = contador['Contador Ruta Revolución'].astype('int64')
contador['Contador Ruta Patriotismo'] = contador['Contador Ruta Patriotismo'].astype('int64')

In [15]:
contador['suma_total'] = contador['Contador Ruta Patriotismo'] + contador['Contador Ruta Revolución'] + contador['Contador Ruta Reforma222']

In [16]:
contador_final = contador

In [17]:
contador_final.head()

,Día de la semana,Día,Mes,Año,Fecha,Contador Ruta Reforma222,Contador Ruta Revolución,Contador Ruta Patriotismo,suma_total
0,sabado,16,noviembre,2013,2013-11-16,613,0,0,613
1,domingo,17,noviembre,2013,2013-11-17,1084,0,0,1084
2,lunes,18,noviembre,2013,2013-11-18,1073,0,0,1073
3,martes,19,noviembre,2013,2013-11-19,1344,0,0,1344
4,miercoles,20,noviembre,2013,2013-11-20,2394,0,0,2394


In [18]:
contador_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2374 entries, 0 to 2373
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Día de la semana           2374 non-null   object        
 1   Día                        2374 non-null   int64         
 2   Mes                        2374 non-null   object        
 3   Año                        2374 non-null   int64         
 4   Fecha                      2374 non-null   datetime64[ns]
 5   Contador Ruta Reforma222   2374 non-null   int64         
 6   Contador Ruta Revolución   2374 non-null   int64         
 7   Contador Ruta Patriotismo  2374 non-null   int64         
 8   suma_total                 2374 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 167.0+ KB


In [19]:
contador_final.to_csv('Data/contador_final.csv', index=False)

# Incidentes ciclistas

In [20]:
incidentes = incidentes[['dia_semana', 'fecha_creacion', 'hora_creacion', 'mes', 'delegacion_inicio', 'latitud', 'longitud', 'geopoint']]

In [21]:
incidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924 entries, 0 to 2923
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dia_semana         2924 non-null   object 
 1   fecha_creacion     2924 non-null   object 
 2   hora_creacion      2924 non-null   object 
 3   mes                2924 non-null   int64  
 4   delegacion_inicio  2924 non-null   object 
 5   latitud            2924 non-null   float64
 6   longitud           2924 non-null   float64
 7   geopoint           2924 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 182.9+ KB


In [22]:
incidentes['dia_semana'] = incidentes['dia_semana'].str.lower()

In [23]:
# Regex matches
r = re.compile(r'dddd')
incidentes.loc[incidentes['hora_creacion'].str.match(r'\d\d')==False, 'hora_creacion'] = '00:00:00'

In [24]:
fecha_hora = pd.Series(incidentes['fecha_creacion'] + ' ' + incidentes['hora_creacion'])

In [25]:
incidentes.insert(1, 'fecha_hora', fecha_hora)

In [26]:
incidentes['fecha_hora'] = pd.to_datetime(incidentes['fecha_hora'], format='%d/%m/%Y %H:%M:%S')

In [27]:
incidentes['fecha_creacion'] = pd.to_datetime(incidentes['fecha_creacion'], format='%d/%m/%Y')

In [28]:
incidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924 entries, 0 to 2923
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   dia_semana         2924 non-null   object        
 1   fecha_hora         2924 non-null   datetime64[ns]
 2   fecha_creacion     2924 non-null   datetime64[ns]
 3   hora_creacion      2924 non-null   object        
 4   mes                2924 non-null   int64         
 5   delegacion_inicio  2924 non-null   object        
 6   latitud            2924 non-null   float64       
 7   longitud           2924 non-null   float64       
 8   geopoint           2924 non-null   object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(4)
memory usage: 205.7+ KB


In [29]:
incidentes.head()

,dia_semana,fecha_hora,fecha_creacion,hora_creacion,mes,delegacion_inicio,latitud,longitud,geopoint
0,miércoles,2019-10-16 09:04:53,2019-10-16,09:04:53,10,VENUSTIANO CARRANZA,19.44961,-99.11384,"19.44961002,-99.11383992"
1,miércoles,2019-10-16 13:35:39,2019-10-16,13:35:39,10,IZTAPALAPA,19.36409,-99.06687,"19.36409004,-99.06687"
2,miércoles,2019-10-16 17:35:24,2019-10-16,17:35:24,10,CUAUHTEMOC,19.42287,-99.12929,"19.42287003,-99.12929004"
3,jueves,2019-10-17 11:27:11,2019-10-17,11:27:11,10,CUAUHTEMOC,19.43451,-99.14268,"19.43451,-99.14267988"
4,viernes,2019-10-18 18:11:00,2019-10-18,18:11:00,10,BENITO JUAREZ,19.38966,-99.18504,"19.38966003,-99.18504"


In [30]:
incidentes['dia_semana'] = incidentes['dia_semana'].str.replace('á', 'a').str.replace('é', 'e')

In [31]:
incidentes['hora'] = incidentes['fecha_hora'].dt.hour
incidentes['año'] = incidentes['fecha_hora'].dt.year
incidentes['mes'] = incidentes['fecha_hora'].dt.month

In [32]:
dict_meses

{'enero': 1,
 'febrero': 2,
 'marzo': 3,
 'abril': 4,
 'mayo': 5,
 'junio': 6,
 'julio': 7,
 'agosto': 8,
 'septiembre': 9,
 'octubre': 10,
 'noviembre': 11,
 'diciembre': 12}

In [33]:
incidentes['mes'] = incidentes['mes'].map(dict_meses_inverso)

In [34]:
incidentes_final = incidentes

In [35]:
incidentes_final.to_csv('Data/incidentes_final.csv', index=False)

## Estudio 2018 Ciclismo

In [36]:
pd.set_option('display.max_columns', 500)

In [37]:
columnas = ['CALLE_LEV', 'F1', 'P1', 'P3', 'P6', 'P15', 'P18', 'P19', 'P20', 'P21_NIVEL', 'P23_1']
nombres = ['calle_lev', 'edad', 'origen', 'destino', 'motivo_viaje', 'motivo_dejar_bici', 'robado_bici', 'robo_forma', 'sexo', 'educacion', 'ingreso_mensual']

In [38]:
estudio1 = estudio[columnas]

In [39]:
ch_columnas = estudio1.columns.tolist()
res = {ch_columnas[i]: nombres[i] for i in range(len(ch_columnas))}

In [40]:
estudio1 = estudio1.rename(columns=res)

In [41]:
dict_sexo = {1: 'Hombre', 2:'Mujer'}
dict_robado_bici = {1:'Sí', 2:'No', 8:'No sabe', 9:'No contestó'}
dict_robo_forma = {1:'Estacionada', 2:'En movimiento', 8:'No sabe', 9:'No contestó'}
dict_motivo_dejar_bici = {1:'Lesión o incapacidad',
                         2:'Fala mecánica de bici',
                         3: 'Cambio de trabajo',
                         4: 'Llevar prisa',
                         5: 'Recorrido muy largo',
                         6: 'Falta de infraestructura',
                         7: 'Clima',
                         8: 'Inseguridad',
                         9: 'No haya bici disponible',
                         10: 'Ningún motivo',
                         11: 'No sé',
                         12: 'No contestó'}

In [42]:
estudio1['sexo'] = estudio1['sexo'].map(dict_sexo)
estudio1['robado_bici'] = estudio1['robado_bici'].map(dict_robado_bici)
estudio1['robo_forma'] = estudio1['robo_forma'].map(dict_robo_forma)
estudio1['motivo_dejar_bici'] = estudio1['motivo_dejar_bici'].map(dict_motivo_dejar_bici)

In [43]:
estudio1.head() 

,calle_lev,edad,origen,destino,motivo_viaje,motivo_dejar_bici,robado_bici,robo_forma,sexo,educacion,ingreso_mensual
0,CAMINO VIEJO A MIXCOAC,48,2,1,2,Cambio de trabajo,Sí,Estacionada,Hombre,4,3
1,VASCO DE QUIROGA,35,1,3,9,Ningún motivo,No,NaN,Hombre,9,6
2,VASCO DE QUIROGA,43,2,2,1,Ningún motivo,No,NaN,Hombre,3,4
3,POSTA,54,1,7,9,Cambio de trabajo,No,NaN,Hombre,4,4
4,CALZADA DE LOS LEONES,35,1,4,4,Ningún motivo,No,NaN,Hombre,8,5


In [44]:
estudio_final = estudio1

In [45]:
estudio_final.to_csv('estudio_final.csv', index=False)

## Motivo de que dejarían bici

In [46]:
a = pd.Series(estudio_final['motivo_dejar_bici'].value_counts())

In [47]:
a.to_csv('Data/motivo_dejar_final.csv')